In [33]:
import numpy as np
import math

class conv2d:
    def __init__(self, k, s, p):
        self.k = k
        self.p = p
        self.s = s

    def get_x(self):
        return self.X + self.p * 2

    def get_y(self):
        return self.Y + self.p * 2

    def get_slides_done(self):# Find slides that are done already
        cols_done = self.y_id * self.y_size + self.p
        h_slides_done = max(math.floor((cols_done-self.k)/self.s) + 1, 0)
        rows_done = self.x_id * self.x_size + self.p
        v_slides_done = max(math.floor((rows_done-self.k)/self.s) + 1, 0)

        # Windows done so far
        return (v_slides_done, h_slides_done)

    def get_block_win_val(self, w, i, j, ch, imgs, img):
        cols_done = self.y_id * self.y_size + self.p
        rows_done = self.x_id * self.x_size + self.p

        h_slides = math.floor((self.get_y()-self.k)/self.s) + 1
        v_slides = math.floor((self.get_x()-self.k)/self.s) + 1

        r = math.floor(w / h_slides)
        c = w % h_slides
        x = (r * self.s) + i
        y = (c * self.s) + j

        print("ORIGIN: ", r * self.s ,c * self.s , "COORD:", x,y, "TCOORD: ", x-rows_done, y-cols_done)
        x -= rows_done
        y -= cols_done

        if x < 0 or y < 0 or x >= self.b_X or y >= self.b_Y:
            return 0

        return imgs[img][ch][x][y]

    def img_block_to_conv2d(self, imgs, X, Y, x_id, y_id, x_size, y_size):
        self.channels, self.b_X, self.b_Y = imgs[0].shape
        self.X, self.Y, self.x_id, self.y_id, self.x_size, self.y_size = X, Y, x_id, y_id, x_size, y_size
        
        cols_done = self.y_id * self.y_size
        rows_done = self.x_id * self.x_size

        h_slides = math.floor((self.get_y()-self.k)/self.s) + 1
        v_slides = math.floor((self.get_x()-self.k)/self.s) + 1

        v_cols = cols_done + self.b_Y + self.p
        v_rows = rows_done + self.b_X + self.p
        if cols_done + self.b_Y >= Y:
            v_cols += self.p
        if rows_done + self.b_X >= X:
            v_rows += self.p

        h_vslides = max(math.floor((v_cols - self.k)/self.s) + 1, 0)
        v_vslides = max(math.floor((v_rows - self.k)/self.s) + 1, 0)
        
        if cols_done + self.b_Y < Y:
            diff = math.ceil((v_cols - (h_vslides - 1) * self.s) / self.s) - 1
            h_vslides += diff
        if rows_done + self.b_X < X:
            diff = math.ceil((v_rows - (v_vslides - 1) * self.s) / self.s) - 1
            v_vslides += diff
        
        print("Virtual matrix slides ", h_vslides, v_vslides)

        v_done, h_done = self.get_slides_done()
        print("Starting window ", v_done, h_done)

        data = {}
        for x in range(v_done, v_vslides):
            for y in range(h_done, h_vslides):
                print("Looking at window ", x * h_slides + y)
                row = []
                for c in range(self.channels):
                    for i in range(self.k):
                        for j in range(self.k):
                            row.append(self.get_block_win_val(x * h_slides + y, i, j, c, imgs, 0))
                data[x * h_slides+ y] = np.array(row)

        return data
 
    def aggregate(self, datas):
        fin = {}
        for partial in datas:
            for window in partial:
                if window in fin:
#                     for i in range(len(fin[window])):
#                         if fin[window][i] == 0:
#                             fin[window][i] = partial[window][i]
#                         else:
#                             assert fin[window][i] == partial[window][i]
                    fin[window] += partial[window]
                else:
                    fin[window] = partial[window]
        res = [fin[d] for d in range(len(fin))]
        return np.array(res)

    def get_win_val(self, w, i, j, ch, imgs, img):    
        h_slides = math.floor((self.get_y()-self.k)/self.s) + 1
        v_slides = math.floor((self.get_x()-self.k)/self.s) + 1

        r = math.floor(w / h_slides)
        c = w % h_slides
        x = (r * self.s) + i - self.p
        y = (c * self.s) + j - self.p

        print("ORIGIN: ", r * self.s ,c * self.s , "COORD:", x,y)
        if x < 0 or y < 0 or x >= self.X or y >= self.Y:
            return 0

        return imgs[img][ch][x][y]
    
    def img_to_conv2d(self, imgs):
        self.channels, self.X, self.Y = imgs[0].shape

        h_slides = math.floor((self.get_y()-self.k)/self.s) + 1
        v_slides = math.floor((self.get_x()-self.k)/self.s) + 1

        windows = h_slides * v_slides

        data = []

        for w in range(windows):
            print("Looking at window ", w)
            row = []
            for c in range(self.channels):
                for i in range(self.k):
                    for j in range(self.k):
                        row.append(self.get_win_val(w, i, j, c, imgs, 0))
            data.append(row)

        return np.array(data)

In [62]:
imgs = np.random.rand(1,1,10,10)

kernel_size = 3
stride = 4
padding = 0

con = conv2d(kernel_size, stride, padding)
data = con.img_to_conv2d(imgs)
print(data.shape)
for i in range(data.shape[0]):
    print(data[i].tolist())

Looking at window  0
ORIGIN:  0 0 COORD: 0 0
ORIGIN:  0 0 COORD: 0 1
ORIGIN:  0 0 COORD: 0 2
ORIGIN:  0 0 COORD: 1 0
ORIGIN:  0 0 COORD: 1 1
ORIGIN:  0 0 COORD: 1 2
ORIGIN:  0 0 COORD: 2 0
ORIGIN:  0 0 COORD: 2 1
ORIGIN:  0 0 COORD: 2 2
Looking at window  1
ORIGIN:  0 4 COORD: 0 4
ORIGIN:  0 4 COORD: 0 5
ORIGIN:  0 4 COORD: 0 6
ORIGIN:  0 4 COORD: 1 4
ORIGIN:  0 4 COORD: 1 5
ORIGIN:  0 4 COORD: 1 6
ORIGIN:  0 4 COORD: 2 4
ORIGIN:  0 4 COORD: 2 5
ORIGIN:  0 4 COORD: 2 6
Looking at window  2
ORIGIN:  4 0 COORD: 4 0
ORIGIN:  4 0 COORD: 4 1
ORIGIN:  4 0 COORD: 4 2
ORIGIN:  4 0 COORD: 5 0
ORIGIN:  4 0 COORD: 5 1
ORIGIN:  4 0 COORD: 5 2
ORIGIN:  4 0 COORD: 6 0
ORIGIN:  4 0 COORD: 6 1
ORIGIN:  4 0 COORD: 6 2
Looking at window  3
ORIGIN:  4 4 COORD: 4 4
ORIGIN:  4 4 COORD: 4 5
ORIGIN:  4 4 COORD: 4 6
ORIGIN:  4 4 COORD: 5 4
ORIGIN:  4 4 COORD: 5 5
ORIGIN:  4 4 COORD: 5 6
ORIGIN:  4 4 COORD: 6 4
ORIGIN:  4 4 COORD: 6 5
ORIGIN:  4 4 COORD: 6 6
(4, 9)
[0.9517326822172918, 0.12799320996871866, 0.6

In [50]:
b_0_0 = np.array(imgs[0][0][:5,:5]).reshape(1,1,5,5)
b_0_1 = np.array(imgs[0][0][:5,5:]).reshape(1,1,5,5)
b_1_0 = np.array(imgs[0][0][5:,:5]).reshape(1,1,5,5)
b_1_1 = np.array(imgs[0][0][5:,5:]).reshape(1,1,5,5)

In [51]:
con = conv2d(kernel_size, stride, padding)
data1 = con.img_block_to_conv2d(b_0_0, 10, 10, 0, 0, 5, 5)
data2 = con.img_block_to_conv2d(b_0_1, 10, 10, 0, 1, 5, 5)
data3 = con.img_block_to_conv2d(b_1_0, 10, 10, 1, 0, 5, 5)
data4 = con.img_block_to_conv2d(b_1_1, 10, 10, 1, 1, 5, 5)


Virtual matrix slides  7 7
Starting window  0 0
Looking at window  0
ORIGIN:  0 0 COORD: 0 0 TCOORD:  -2 -2
ORIGIN:  0 0 COORD: 0 1 TCOORD:  -2 -1
ORIGIN:  0 0 COORD: 0 2 TCOORD:  -2 0
ORIGIN:  0 0 COORD: 1 0 TCOORD:  -1 -2
ORIGIN:  0 0 COORD: 1 1 TCOORD:  -1 -1
ORIGIN:  0 0 COORD: 1 2 TCOORD:  -1 0
ORIGIN:  0 0 COORD: 2 0 TCOORD:  0 -2
ORIGIN:  0 0 COORD: 2 1 TCOORD:  0 -1
ORIGIN:  0 0 COORD: 2 2 TCOORD:  0 0
Looking at window  1
ORIGIN:  0 1 COORD: 0 1 TCOORD:  -2 -1
ORIGIN:  0 1 COORD: 0 2 TCOORD:  -2 0
ORIGIN:  0 1 COORD: 0 3 TCOORD:  -2 1
ORIGIN:  0 1 COORD: 1 1 TCOORD:  -1 -1
ORIGIN:  0 1 COORD: 1 2 TCOORD:  -1 0
ORIGIN:  0 1 COORD: 1 3 TCOORD:  -1 1
ORIGIN:  0 1 COORD: 2 1 TCOORD:  0 -1
ORIGIN:  0 1 COORD: 2 2 TCOORD:  0 0
ORIGIN:  0 1 COORD: 2 3 TCOORD:  0 1
Looking at window  2
ORIGIN:  0 2 COORD: 0 2 TCOORD:  -2 0
ORIGIN:  0 2 COORD: 0 3 TCOORD:  -2 1
ORIGIN:  0 2 COORD: 0 4 TCOORD:  -2 2
ORIGIN:  0 2 COORD: 1 2 TCOORD:  -1 0
ORIGIN:  0 2 COORD: 1 3 TCOORD:  -1 1
ORIGIN:  0 2

In [52]:
d = con.aggregate([data1, data2, data3, data4])
for i in range(d.shape[0]):
    print(d[i].tolist())

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3708332087323861]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3708332087323861, 0.2966087815780607]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3708332087323861, 0.2966087815780607, 0.4276516658656625]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2966087815780607, 0.4276516658656625, 0.8367798163652478]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4276516658656625, 0.8367798163652478, 0.7808629925657536]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8367798163652478, 0.7808629925657536, 0.9877812489756163]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7808629925657536, 0.9877812489756163, 0.7286657402939747]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9877812489756163, 0.7286657402939747, 0.35135656854081954]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7286657402939747, 0.35135656854081954, 0.6945914010953669]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.35135656854081954, 0.6945914010953669, 0.23403059033115536]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6945914010953669, 0.23403059033115536, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.23403059033115536, 0.

In [53]:
np.array_equal(data,d)

True

In [56]:
imgs = np.random.rand(1,1,12,12)

kernel_size = 3
stride = 1
padding = 2

con = conv2d(kernel_size, stride, padding)
data = con.img_to_conv2d(imgs)
print(data.shape)
for i in range(data.shape[0]):
    print(data[i].tolist())

Looking at window  0
ORIGIN:  0 0 COORD: -2 -2
ORIGIN:  0 0 COORD: -2 -1
ORIGIN:  0 0 COORD: -2 0
ORIGIN:  0 0 COORD: -1 -2
ORIGIN:  0 0 COORD: -1 -1
ORIGIN:  0 0 COORD: -1 0
ORIGIN:  0 0 COORD: 0 -2
ORIGIN:  0 0 COORD: 0 -1
ORIGIN:  0 0 COORD: 0 0
Looking at window  1
ORIGIN:  0 1 COORD: -2 -1
ORIGIN:  0 1 COORD: -2 0
ORIGIN:  0 1 COORD: -2 1
ORIGIN:  0 1 COORD: -1 -1
ORIGIN:  0 1 COORD: -1 0
ORIGIN:  0 1 COORD: -1 1
ORIGIN:  0 1 COORD: 0 -1
ORIGIN:  0 1 COORD: 0 0
ORIGIN:  0 1 COORD: 0 1
Looking at window  2
ORIGIN:  0 2 COORD: -2 0
ORIGIN:  0 2 COORD: -2 1
ORIGIN:  0 2 COORD: -2 2
ORIGIN:  0 2 COORD: -1 0
ORIGIN:  0 2 COORD: -1 1
ORIGIN:  0 2 COORD: -1 2
ORIGIN:  0 2 COORD: 0 0
ORIGIN:  0 2 COORD: 0 1
ORIGIN:  0 2 COORD: 0 2
Looking at window  3
ORIGIN:  0 3 COORD: -2 1
ORIGIN:  0 3 COORD: -2 2
ORIGIN:  0 3 COORD: -2 3
ORIGIN:  0 3 COORD: -1 1
ORIGIN:  0 3 COORD: -1 2
ORIGIN:  0 3 COORD: -1 3
ORIGIN:  0 3 COORD: 0 1
ORIGIN:  0 3 COORD: 0 2
ORIGIN:  0 3 COORD: 0 3
Looking at window  

In [57]:
b_0_0 = np.array(imgs[0][0][0:3,0:3]).reshape(1,1,3,3)
b_0_1 = np.array(imgs[0][0][0:3,3:6]).reshape(1,1,3,3)
b_0_2 = np.array(imgs[0][0][0:3,6:9]).reshape(1,1,3,3)
b_0_3 = np.array(imgs[0][0][0:3,9:12]).reshape(1,1,3,3)

b_1_0 = np.array(imgs[0][0][3:6,0:3]).reshape(1,1,3,3)
b_1_1 = np.array(imgs[0][0][3:6,3:6]).reshape(1,1,3,3)
b_1_2 = np.array(imgs[0][0][3:6,6:9]).reshape(1,1,3,3)
b_1_3 = np.array(imgs[0][0][3:6,9:12]).reshape(1,1,3,3)

b_2_0 = np.array(imgs[0][0][6:9,0:3]).reshape(1,1,3,3)
b_2_1 = np.array(imgs[0][0][6:9,3:6]).reshape(1,1,3,3)
b_2_2 = np.array(imgs[0][0][6:9,6:9]).reshape(1,1,3,3)
b_2_3 = np.array(imgs[0][0][6:9,9:12]).reshape(1,1,3,3)

b_3_0 = np.array(imgs[0][0][9:12,0:3]).reshape(1,1,3,3)
b_3_1 = np.array(imgs[0][0][9:12,3:6]).reshape(1,1,3,3)
b_3_2 = np.array(imgs[0][0][9:12,6:9]).reshape(1,1,3,3)
b_3_3 = np.array(imgs[0][0][9:12,9:12]).reshape(1,1,3,3)

In [58]:
con = conv2d(kernel_size, stride, padding)

data00 = con.img_block_to_conv2d(b_0_0, 12, 12, 0, 0, 3, 3)
data01 = con.img_block_to_conv2d(b_0_1, 12, 12, 0, 1, 3, 3)
data02 = con.img_block_to_conv2d(b_0_2, 12, 12, 0, 2, 3, 3)
data03 = con.img_block_to_conv2d(b_0_3, 12, 12, 0, 3, 3, 3)

data10 = con.img_block_to_conv2d(b_1_0, 12, 12, 1, 0, 3, 3)
data11 = con.img_block_to_conv2d(b_1_1, 12, 12, 1, 1, 3, 3)
data12 = con.img_block_to_conv2d(b_1_2, 12, 12, 1, 2, 3, 3)
data13 = con.img_block_to_conv2d(b_1_3, 12, 12, 1, 3, 3, 3)

data20 = con.img_block_to_conv2d(b_2_0, 12, 12, 2, 0, 3, 3)
data21 = con.img_block_to_conv2d(b_2_1, 12, 12, 2, 1, 3, 3)
data22 = con.img_block_to_conv2d(b_2_2, 12, 12, 2, 2, 3, 3)
data23 = con.img_block_to_conv2d(b_2_3, 12, 12, 2, 3, 3, 3)

data30 = con.img_block_to_conv2d(b_3_0, 12, 12, 3, 0, 3, 3)
data31 = con.img_block_to_conv2d(b_3_1, 12, 12, 3, 1, 3, 3)
data32 = con.img_block_to_conv2d(b_3_2, 12, 12, 3, 2, 3, 3)
data33 = con.img_block_to_conv2d(b_3_3, 12, 12, 3, 3, 3, 3)

Virtual matrix slides  5 5
Starting window  0 0
Looking at window  0
ORIGIN:  0 0 COORD: 0 0 TCOORD:  -2 -2
ORIGIN:  0 0 COORD: 0 1 TCOORD:  -2 -1
ORIGIN:  0 0 COORD: 0 2 TCOORD:  -2 0
ORIGIN:  0 0 COORD: 1 0 TCOORD:  -1 -2
ORIGIN:  0 0 COORD: 1 1 TCOORD:  -1 -1
ORIGIN:  0 0 COORD: 1 2 TCOORD:  -1 0
ORIGIN:  0 0 COORD: 2 0 TCOORD:  0 -2
ORIGIN:  0 0 COORD: 2 1 TCOORD:  0 -1
ORIGIN:  0 0 COORD: 2 2 TCOORD:  0 0
Looking at window  1
ORIGIN:  0 1 COORD: 0 1 TCOORD:  -2 -1
ORIGIN:  0 1 COORD: 0 2 TCOORD:  -2 0
ORIGIN:  0 1 COORD: 0 3 TCOORD:  -2 1
ORIGIN:  0 1 COORD: 1 1 TCOORD:  -1 -1
ORIGIN:  0 1 COORD: 1 2 TCOORD:  -1 0
ORIGIN:  0 1 COORD: 1 3 TCOORD:  -1 1
ORIGIN:  0 1 COORD: 2 1 TCOORD:  0 -1
ORIGIN:  0 1 COORD: 2 2 TCOORD:  0 0
ORIGIN:  0 1 COORD: 2 3 TCOORD:  0 1
Looking at window  2
ORIGIN:  0 2 COORD: 0 2 TCOORD:  -2 0
ORIGIN:  0 2 COORD: 0 3 TCOORD:  -2 1
ORIGIN:  0 2 COORD: 0 4 TCOORD:  -2 2
ORIGIN:  0 2 COORD: 1 2 TCOORD:  -1 0
ORIGIN:  0 2 COORD: 1 3 TCOORD:  -1 1
ORIGIN:  0 2

In [59]:
d = con.aggregate([data00, data01, data02, data03, data10, data11, data12, data13, data20, data21, data22, data23, data30, data31, data32, data33])
for i in range(d.shape[0]):
    print(d[i].tolist())

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0794990503568479]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0794990503568479, 0.10374370710739345]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0794990503568479, 0.10374370710739345, 0.2570658278773227]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.10374370710739345, 0.2570658278773227, 0.7369902439653028]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2570658278773227, 0.7369902439653028, 0.5332788083694948]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7369902439653028, 0.5332788083694948, 0.36369133035136725]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5332788083694948, 0.36369133035136725, 0.6945360847977217]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.36369133035136725, 0.6945360847977217, 0.17351352560533462]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6945360847977217, 0.17351352560533462, 0.7950667724241544]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.17351352560533462, 0.7950667724241544, 0.5807494208981656]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7950667724241544, 0.5807494208981656, 0.6036041411266199]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.58

In [60]:
np.array_equal(d,data)

True